## A lot of websites support REST API to query data, and in your daily job you will use it very often. RESTful API = REpresentational State Transfer. 

Key points of REST API: 

1. Client-Server
2. Stateless
3. Cacheable
4. Uniform-interface
5. Layered System
6. Code on demond

## REST API is a preferred way to pull data from websites using crawlers
1. You can get structured data directly, and easily parse them into dataframe
2. It's much more stable than a web crawler, website can update front page frequently but will always keep the API query the same syntax/format, or have compatibility support -- a lot of apps depend on the reliability of their API service
3. It's more cost effective to both you and the web server
4. It allows to query some data unseen from web page e.g. stock price at tick level

## REST API consist of: a base URL + standard HTTP method (Get, Put, Post, Delete) + a media type (mostly JSON or XML)

In python, we can use 
1. requests to send/receive HTTP request from client side
2. json to serialize and deserialize data 
3. flask to build up web server to response to HTTP request

Let's dive in with an example from Yelp.

Go to https://www.yelp.com/developers/documentation/v3/business_search (you may need to sign up Yelp developer's account to get access to these documentations)
Read through this page, and understand
1. What HTTP method should be used?
2. What is the basic endpoint?
3. What is the format of the response body?
4. Can you figure out how to search for restaurants within 10 miles of zip code 94583?

Now go to https://www.yelp.com/developers/v3/manage_app  to add an app to your developer's account, get your client ID and app key, which will be used later. The default daily request limit is 5000.

In [1]:
#pip install requests if you don't already have it
import requests
#test if your request is working fine, you should be getting a status code of 200 from below command
response = requests.get('https://api.github.com')
print(response)

<Response [200]>


Some common HTTP response code:
200: Everything OK
400: Bad request, check your request syntax
401/403: Unathorized, you may need permission
404: Page not found, you know it
405: Bad method
500: Internal Server Error, check your server code

A HTTP request example:

POST /cgi-bin/process.cgi HTTP/1.1
User-Agent: Mozilla/4.0 (compatible; MSIE5.01; Windows NT)
Host: www.tutorialspoint.com
Content-Type: application/x-www-form-urlencoded
Content-Length: length
Accept-Language: en-us
Accept-Encoding: gzip, deflate
Connection: Keep-Alive

licenseID=string&content=string&/paramsXML=string

In [9]:
#let's try to send GET request to Yelp, replace the API_Key with yours
API_Key = 'USE YOUR keys'
url = 'https://api.yelp.com/v3/businesses/search'
#'location':'22182',
params = {
    'term':'Restaurants',
    'location':'94555',
    'radius':'16000',
    'limit':'4'
}
headers = {
    'Authorization': 'Bearer %s' % API_Key,
    'Content-Type':'application/json'
}
#the browser needs full url to work, requests helps you to automatically format those urls
#e.g. https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Ramon%2C+CA&ns=1
response = requests.get(url, params=params, headers=headers, json=None) 
#GET request usually don't send data to server, so json=None. If you need to POST data, you can use json={'key':'value'}

In [10]:
print(response)

<Response [200]>


In [13]:
# to view the contents of the response
response.content

b'{"businesses": [{"id": "G_qFJBwdgRqjjrapidHPwg", "alias": "fong-kee-clay-pot-union-city", "name": "Fong Kee Clay Pot", "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/n-llcDFpxpwOiXEMGLi6WA/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/fong-kee-clay-pot-union-city?adjust_creative=Eoyi_YSkAfM5aeQla0x-3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Eoyi_YSkAfM5aeQla0x-3A", "review_count": 5, "categories": [{"alias": "cantonese", "title": "Cantonese"}], "rating": 5.0, "coordinates": {"latitude": 37.58947, "longitude": -122.02283}, "transactions": [], "location": {"address1": "1773 Decoto Rd", "address2": null, "address3": "", "city": "Union City", "zip_code": "94587", "country": "US", "state": "CA", "display_address": ["1773 Decoto Rd", "Union City, CA 94587"]}, "phone": "+15109529093", "display_phone": "(510) 952-9093", "distance": 6484.68764409451}, {"id": "purj1aiUzDi0I__qLOaNRg", "alias": "pan-fried-dumplings-newark", "name": "Pan Fried 

In [14]:
#other properties and methods of response object, a full list of available properties and methods can be found here
#https://www.w3schools.com/python/ref_requests_response.asp
print('cookies is ' + str(response.cookies) + '\n')
print('headers is ' + str(response.headers) + '\n')
print('response body is ' + str(response.text))

cookies is <RequestsCookieJar[]>

headers is {'Connection': 'keep-alive', 'content-type': 'application/json', 'x-routing-service': 'routing-main--uswest2-b56d7b5c4-4kz4r; site=public_api_v3', 'ratelimit-dailylimit': '5000', 'ratelimit-remaining': '4999', 'ratelimit-resettime': '2022-08-04T00:00:00+00:00', 'server': 'envoy', 'x-b3-sampled': '0', 'x-zipkin-id': '4f777cc2f0d96b75', 'x-cloudmap': 'routing_uswest2', 'x-mode': 'ro', 'x-proxied': '10-69-184-36-uswest2bprod', 'content-encoding': 'gzip', 'x-extlb': '10-69-184-36-uswest2bprod', 'cache-control': 'max-age=0, no-store, private, no-transform', 'Accept-Ranges': 'bytes', 'Date': 'Wed, 03 Aug 2022 07:03:12 GMT', 'Via': '1.1 varnish', 'X-Served-By': 'cache-pao17461-PAO', 'X-Cache': 'MISS', 'X-Cache-Hits': '0', 'Vary': 'Accept-Encoding', 'transfer-encoding': 'chunked'}

response body is {"businesses": [{"id": "G_qFJBwdgRqjjrapidHPwg", "alias": "fong-kee-clay-pot-union-city", "name": "Fong Kee Clay Pot", "image_url": "https://s3-media3.fl

In [15]:
#response.json() can automatically parse json text into python dict
#or you can also use json package to parse the text
response.json()

{'businesses': [{'id': 'G_qFJBwdgRqjjrapidHPwg',
   'alias': 'fong-kee-clay-pot-union-city',
   'name': 'Fong Kee Clay Pot',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/n-llcDFpxpwOiXEMGLi6WA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/fong-kee-clay-pot-union-city?adjust_creative=Eoyi_YSkAfM5aeQla0x-3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Eoyi_YSkAfM5aeQla0x-3A',
   'review_count': 5,
   'categories': [{'alias': 'cantonese', 'title': 'Cantonese'}],
   'rating': 5.0,
   'coordinates': {'latitude': 37.58947, 'longitude': -122.02283},
   'transactions': [],
   'location': {'address1': '1773 Decoto Rd',
    'address2': None,
    'address3': '',
    'city': 'Union City',
    'zip_code': '94587',
    'country': 'US',
    'state': 'CA',
    'display_address': ['1773 Decoto Rd', 'Union City, CA 94587']},
   'phone': '+15109529093',
   'display_phone': '(510) 952-9093',
   'distance': 6484.68764409451},
  {'id': 'purj1aiUzDi0I

In [16]:
type(response.json())

dict

In [18]:
#let's take a look at what information you can get
print(response.json().keys())
#And you can easily load it into a dataframe
import pandas as pd
businesses = response.json()['businesses']
df = pd.DataFrame(businesses)
df

dict_keys(['businesses', 'total', 'region'])


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,G_qFJBwdgRqjjrapidHPwg,fong-kee-clay-pot-union-city,Fong Kee Clay Pot,https://s3-media3.fl.yelpcdn.com/bphoto/n-llcD...,False,https://www.yelp.com/biz/fong-kee-clay-pot-uni...,5,"[{'alias': 'cantonese', 'title': 'Cantonese'}]",5.0,"{'latitude': 37.58947, 'longitude': -122.02283}",[],"{'address1': '1773 Decoto Rd', 'address2': Non...",+15109529093,(510) 952-9093,6484.687644,NaN
1,purj1aiUzDi0I__qLOaNRg,pan-fried-dumplings-newark,Pan Fried Dumplings,https://s3-media1.fl.yelpcdn.com/bphoto/3PuNE8...,False,https://www.yelp.com/biz/pan-fried-dumplings-n...,467,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...",4.5,"{'latitude': 37.548733, 'longitude': -122.0493...",[restaurant_reservation],"{'address1': '6180 Jarvis Ave', 'address2': 'S...",+15108941243,(510) 894-1243,2608.304507,$$
2,HUXRBAnq-zruEgE2RFw9sQ,ambrose-butchery-fremont,Ambrose Butchery,https://s3-media2.fl.yelpcdn.com/bphoto/5xZxHG...,False,https://www.yelp.com/biz/ambrose-butchery-frem...,201,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 37.5764345126062, 'longitude': -1...",[delivery],"{'address1': '37683 Niles Blvd', 'address2': '...",+15105092677,(510) 509-2677,9359.532830,$$
3,iKJ8LTC61-QsLTLi1Cte1w,farmhouse-kitchen-thai-cuisine-menlo-park-2,Farmhouse Kitchen Thai Cuisine,https://s3-media3.fl.yelpcdn.com/bphoto/_vDlY-...,False,https://www.yelp.com/biz/farmhouse-kitchen-tha...,799,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 37.45491677842326, 'longitude': -...",[restaurant_reservation],"{'address1': '1165 Merrill St', 'address2': ''...",+16506657935,(650) 665-7935,14174.951535,$$


In [21]:
# save as csv file
df.to_csv('yelp_resturants', index=False)

## Sometimes you need to send data to server, let's take a look at json

JSON stands for JavaScript Object Notation, it's a string representing serielized object/data, contains only data and can easily parse between all kinds of popular data format, e.g. dict, dataframe, csv file, and can be used across all languages.

You will encounter JSON objects and JSON Arrays. e.g.
{
"name":"John",
"age":30,
"cars":[ "Ford", "BMW", "Fiat" ]
}

Conversion table between JSON and Python types:

| JSON | Python |
| --- | --- |
| object | dict |
| array | list |
| string | str |
| number(int/real) | int/float |
| true/false | True/False |

In [19]:
import json

cars = {
    "name":"John",
    "age":"30",
    "cars":["Ford","BMW","Fiat"]
}

print('type of cars is ' + str(type(cars)))

cars_json = json.dumps(cars)
print('type of cars_json is ' + str(type(cars_json)))

cars_from_json = json.loads(cars_json)
print('type of cars_from_json is ' + str(type(cars_from_json)))

type of cars is <class 'dict'>
type of cars_json is <class 'str'>
type of cars_from_json is <class 'dict'>


In [20]:
# you can easily load json string into dataframe also

df_car = pd.read_json(cars_json, orient='columns')
df_car.head()

,name,age,cars
0,John,30,Ford
1,John,30,BMW
2,John,30,Fiat


In [ ]:
## Homework: use github api to create a new repo for yourself